# Set-Up

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3 torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
%cd stylegan2-ada-pytorch/

# Negative Truncation

In [ ]:
!python generate.py --outdir=out --trunc=-1 --seeds=3190102548 --network=/content/drive/MyDrive/Training/best_pkl/final_256.pkl

# Define 5 Random Seeds

In [ ]:
import random

latents = []
for i in range(5):
  latents.append(random.randint(0,2**32-1))

print(latents)

# Save 5 Images of Each Model

In [ ]:
import torch
import numpy as np
import legacy
import PIL.Image
import dnnlib
import os
import shutil

eval_folder_path = "/content/drive/MyDrive/Philip/P~U/SS~2021/Assests/Samples"
model_folder_path = "/content/drive/MyDrive/Training/best_pkl"

truncation_value = 1
device = torch.device('cuda')

for model in os.listdir(model_folder_path):

  model_path = os.path.join(model_folder_path, model)
  eval_path = os.path.join(eval_folder_path, os.path.splitext(model)[0])

  try:
    shutil.rmtree(eval_path)
  except:
    print("Missing folder created")
  os.makedirs(eval_path, exist_ok=True)

  with dnnlib.util.open_url(model_path) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)

  for l in latents:
    z = torch.from_numpy(np.random.RandomState(l).randn(1, G.z_dim)).to(device)
    label = torch.zeros([1, G.c_dim], device=device)

    img = G(z, label, truncation_psi=truncation_value, noise_mode="const")
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{eval_path}/seed{l}.png')

# Save The Average Image of Each Model

In [ ]:
import torch
import numpy as np
import legacy
import PIL.Image
import dnnlib
import os
import shutil

eval_folder_path = "/content/drive/MyDrive/Philip/P~U/SS~2021/Assests/SamplesTrunc0"
model_folder_path = "/content/drive/MyDrive/Training/best_pkl"

truncation_value = 0
device = torch.device('cuda')

for model in os.listdir(model_folder_path):

  model_path = os.path.join(model_folder_path, model)
  eval_path = os.path.join(eval_folder_path, os.path.splitext(model)[0])

  try:
    shutil.rmtree(eval_path)
  except:
    print("Missing folder created")
  os.makedirs(eval_path, exist_ok=True)

  with dnnlib.util.open_url(model_path) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)

  l = random.randint(0,2**32-1)
  z = torch.from_numpy(np.random.RandomState(l).randn(1, G.z_dim)).to(device)
  label = torch.zeros([1, G.c_dim], device=device)

  img = G(z, label, truncation_psi=truncation_value, noise_mode="const")
  img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{eval_path}/seed{l}.png')

# Save 50 Images of One Model

In [ ]:
# Save 50 images of one model

import torch
import numpy as np
import legacy
import PIL.Image
import dnnlib
import os
import shutil
import random

latents = []
for i in range(500):
  latents.append(random.randint(0,2**32-1))

print(latents)

eval_folder_path = "/content/drive/MyDrive/Philip/P~U/SS~2021/Assests/256_final_samples"
model_folder_path = "/content/drive/MyDrive/Training/best_pkl/final_256.pkl"

truncation_value = 1
device = torch.device('cuda')

try:
  shutil.rmtree(eval_folder_path)
except:
  print("Missing folder created")
os.makedirs(eval_folder_path, exist_ok=True)

with dnnlib.util.open_url(model_folder_path) as f:
  G = legacy.load_network_pkl(f)['G_ema'].to(device)

for l in latents:
  z = torch.from_numpy(np.random.RandomState(l).randn(1, G.z_dim)).to(device)
  label = torch.zeros([1, G.c_dim], device=device)

  img = G(z, label, truncation_psi=truncation_value, noise_mode="const")
  img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{eval_folder_path}/seed{l}.png')

# Style-Mixing

In [ ]:
# styles 0-13, 0-1 2-5 6-13
!python style_mixing.py --styles=6-13 --outdir=/content/drive/MyDrive/Philip/P~U/SS~2021/Assests/6-13 --rows=4264001345,4229487283,3697211462,3432756122,3310702129,3190102548 --cols=4264001345,4229487283,3697211462,3432756122,3310702129,3190102548 --network=/content/drive/MyDrive/Training/best_pkl/final_256.pkl

# Create FID Development Plots

In [ ]:
import os
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
import math

path_to_model_dir = "/content/drive/MyDrive/Training/"
model_folder_list = os.listdir(path_to_model_dir)

fid_values_all_models = {}

for model_folder in model_folder_list:
  if "training" in model_folder:
    model_folder_path = os.path.join(path_to_model_dir, model_folder)
    run_folder_list = sorted(os.listdir(model_folder_path))

    y = []

    for j, run_folder in enumerate(run_folder_list):
      skip = True

      if j == 0:
        skip = False

      run_folder_path = os.path.join(model_folder_path, run_folder)
      with open(os.path.join(run_folder_path,"metric-fid50k_full.jsonl"), 'r') as json_file:
        json_list = list(json_file)
        for i, json_line in enumerate(json_list):
          if i == 0 and skip:
            continue
          fid_value = json.loads(json_line)["results"]["fid50k_full"]
          y.append(fid_value)

    fid_values_all_models[model_folder] = y

x = np.linspace(0,12.6,64)[:-1]

dataframe = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in fid_values_all_models.items()]))

sns.set()
fig, ax = plt.subplots(figsize=(9,5))

# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_gauto"], ax=ax, label=r"$\gamma = 0,1024$")
# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_g5"], ax=ax, label=r"$\gamma = 5$")
# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_g10"], ax=ax, label=r"$\gamma = 10$")
# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_g15"], ax=ax, label=r"$\gamma = 15$")
# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_g100"], ax=ax, label=r"$\gamma = 100$")
# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_g1000"], ax=ax, label=r"$\gamma = 1000$")

# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_gauto"], ax=ax, label="ADA, no x-flips")
# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_gautonoada"], ax=ax, label="no ADA, no x-flips")
# sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_gautomirror"], ax=ax, label="ADA, x-flips")

sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_gauto"], ax=ax, label=r"31k, $128\times128$")
sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_256_13k"], ax=ax, label=r"13k, $256\times256$")
sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_128_10k"], ax=ax, label=r"11k, $128\times128$")
sns.lineplot(x = x, y = dataframe.loc[:, "training_sets_128_1.9k"], ax=ax, label=r"2k, $128\times128$")

ax.axvline(10.6, color="b", linewidth=1)
ax.set_xlabel("Training progress (number of real images shown to the discriminator)", labelpad=10, fontsize=14)
ax.set_ylabel("FID", labelpad=5, fontsize=14)

######################################
def forward(x):
    return np.log(2+x)


def inverse(x):
    return np.exp(x)-2

ax.set_xscale('function', functions=(forward, inverse))
###############################################

ax.set_yscale("log")
ax.set_yticks([10,15,20,25,30,50,100,200,300])
ax.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
###############################################
plt.xlim(xmin=0)
ax.xaxis.set_major_formatter(FormatStrFormatter('%dM'))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, fontsize=14, loc="upper right")
plt.show()
fig.savefig('fid_figure_ds.png', bbox_inches='tight', pad_inches=0)

# Find The Snapshot With The Lowest FID of Each Model

In [ ]:
lowest = [100, "", 0]

for k,v in fid_values_all_models.items():
  for i, fid in enumerate(v):
    if fid < lowest[0]:
      lowest[0] = fid
      lowest[1] = k
      lowest[2] = i

print(lowest)

# Project Image to Latent Space

In [ ]:
# projection of apple.com
!python projector.py --save-video=False --outdir=/content/drive/MyDrive/Philip/P~U/SS~2021/Assests/Projection/apple10k --target=/content/drive/MyDrive/Training/good_images/apple_com.png --network=/content/drive/MyDrive/Training/best_pkl/10827images_fid14.41.pkl

# Style-Mix of Projected Image With Samples

In [ ]:
import os
import re
from typing import List

import click
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy

def generate_style_mix(
    network_pkl: str,
    row_seeds: List[int],
    col_seeds: List[int],
    col_styles: List[int],
    truncation_psi: float,
    noise_mode: str,
    outdir: str,
    proj_w: str
):

  print('Loading networks from "%s"...' % network_pkl)
  device = torch.device('cuda')
  with dnnlib.util.open_url(network_pkl) as f:
      G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

  os.makedirs(outdir, exist_ok=True)

  ws = np.load(proj_w)['w']
  ws = torch.tensor(ws, device=device)

  print('Generating W vectors...')
  all_seeds = list(set(row_seeds + col_seeds))
  all_z = np.stack([np.random.RandomState(seed).randn(G.z_dim) for seed in all_seeds])
  all_w = G.mapping(torch.from_numpy(all_z).to(device), None)

  w_avg = G.mapping.w_avg
  all_w = w_avg + (all_w - w_avg) * truncation_psi

  for w in ws:
    all_w = torch.cat((all_w, w.unsqueeze(0)),0)

  all_seeds.append("proj_w")
  row_seeds.append("proj_w")
  col_seeds.append("proj_w")

  w_dict = {seed: w for seed, w in zip(all_seeds, list(all_w))}


  print('Generating images...')
  all_images = G.synthesis(all_w, noise_mode=noise_mode)
  all_images = (all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
  image_dict = {(seed, seed): image for seed, image in zip(all_seeds, list(all_images))}

  print('Generating style-mixed images...')
  for row_seed in row_seeds:
      for col_seed in col_seeds:
          w = w_dict[row_seed].clone()
          w[col_styles] = w_dict[col_seed][col_styles]
          image = G.synthesis(w[np.newaxis], noise_mode=noise_mode)
          image = (image.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
          image_dict[(row_seed, col_seed)] = image[0].cpu().numpy()

  print('Saving images...')
  os.makedirs(outdir, exist_ok=True)
  for (row_seed, col_seed), image in image_dict.items():
      PIL.Image.fromarray(image, 'RGB').save(f'{outdir}/{row_seed}-{col_seed}.png')

  print('Saving image grid...')
  W = G.img_resolution
  H = G.img_resolution
  canvas = PIL.Image.new('RGB', (W * (len(col_seeds) + 1), H * (len(row_seeds) + 1)), 'black')
  for row_idx, row_seed in enumerate([0] + row_seeds):
      for col_idx, col_seed in enumerate([0] + col_seeds):
          if row_idx == 0 and col_idx == 0:
              continue
          key = (row_seed, col_seed)
          if row_idx == 0:
              key = (col_seed, col_seed)
          if col_idx == 0:
              key = (row_seed, row_seed)
          canvas.paste(PIL.Image.fromarray(image_dict[key], 'RGB'), (W * col_idx, H * row_idx))
  canvas.save(f'{outdir}/grid.png')

generate_style_mix(
  network_pkl = "/content/drive/MyDrive/Training/best_pkl/final_256.pkl",
  row_seeds = [3432756122,3310702129],
  col_seeds = [3432756122,3310702129],
  col_styles = [2,3,4,5],
  truncation_psi = 1,
  noise_mode = "const",
  outdir = "/content/drive/MyDrive/Philip/P~U/SS~2021/Assests/Projection/style_mix",
  proj_w = "/content/drive/MyDrive/Philip/P~U/SS~2021/Assests/Projection/applefinal/projected_w.npz"
)

# Closed-Form Factorization

In [ ]:
# Closed Form Factorization
!git clone https://github.com/pbizimis/stylegan2-ada-pytorch.git

In [ ]:
%cd stylegan2-ada-pytorch/

In [ ]:
!python closed_form_factorization.py --ckpt /content/drive/MyDrive/Training/best_pkl/final_256.pkl

In [ ]:
!python apply_factor.py -i=152 --truncation=1 --degree=20 --seeds=4264001345,4229487283,3697211462,3432756122,3310702129,3190102548 --ckpt  /content/drive/MyDrive/Training/best_pkl/final_256.pkl factor.pt --no-video --output=/content/drive/MyDrive/Philip/P~U/SS~2021/Assests/CCF